# Multilayer Perceptrons from Scratch

Now that we've covered all the preliminaries, extending to deep neural networks is actually quite easy.

In [108]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np

ctx = mx.cpu()

## MNIST Data (surprise!)

Let's go ahead and grab our data.

In [109]:
mnist = mx.test_utils.get_mnist()
batch_size = 64
train_data = mx.io.NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(mnist["test_data"], mnist["test_label"], batch_size, shuffle=True)

## Multilayer Perceptrons

Here's where things start to get interesting. Before, we mapped our inputs directly onto our outputs through a single linear transformation. 

![](img/simple-softmax-net.png)

This model is perfectly adequate when the underlying relationship between our data points and labels is approximately linear. When our data points and targets are characterized by a more complex relationship, a linear model and produce sucky results. We can model a more general class of functions by incorporating one or more *hidden layers*.

![](img/multilayer-perceptron.png)

Here, each layer will require it's own set of parameters. To make things simple here, we'll assume two hidden layers of computation.

In [110]:
W1 = nd.random_normal(shape=(784,256)) *.01
b1 = nd.random_normal(shape=256) * .01

W2 = nd.random_normal(shape=(256,128)) *.01
b2 = nd.random_normal(shape=128) * .01

W3 = nd.random_normal(shape=(128,10)) *.01
b3 = nd.random_normal(shape=10) *.01

params = [W1, b1, W2, b2, W3, b3]

Again, let's allocate space for gradients.

In [111]:
for param in params:
    param.attach_grad()

## Activation functions

If we compose a multi-layer network but use only linear operations, then our entire network will still be a linear function. That's because $\hat{y} = X \cdot W_1 \cdot W_2 \cdot W_2 = X \cdot W_4 $ for $W_4 = W_1 \cdot W_2 \cdot W3$. To give our model the capacity to capture nonlinear functions, we'll need to interleave our linear operations with activation functions. In this case, we'll use the rectified linear unit (ReLU):

In [112]:
def relu(X):
    return nd.maximum(X,nd.zeros_like(X))

## Softmax Outputs

As with multiclass logistic regression, we'll want out outputs to be *stochastic*, meaning that they constitute a valid probability distribution. We'll use the same softmax activation functino on our output to make sure that our outputs sum to one and are non-negative.




In [113]:
def softmax(y_linear):
    exp = nd.exp(y_linear)
    partition =nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

## Defining the Model

Now we're ready to define our model

In [114]:
def net(X):
    h1_linear = nd.dot(X, W1) + b1
#     print("h1_linear shape: %s" % (np.array(h1_linear.shape)))
    h1 = relu(h1_linear)
#     print("h1 shape: %s" % (np.array(h1.shape)))
    h2_linear = nd.dot(h1, W2) + b2
#     print("h2_linear shape: %s" % (np.array(h2_linear.shape)))
    h2 = relu(h2_linear)
    yhat_linear = nd.dot(h2, W3) + b3
    yhat = softmax(yhat_linear)
#     print(yhat.shape)
    return yhat

## The Cross-entropy Loss Function

Nothing changes here.

In [115]:
def cross_entropy(yhat, y):
    return - nd.sum(y * nd.log(yhat), axis=0, exclude=True)

## Optimizer

...

In [116]:
def SGD(params, lr):    
    for param in params:
        param[:] = param - lr * param.grad

## Evaluation Metric

...

In [117]:
metric = mx.metric.Accuracy()

def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        with autograd.record():
            data = batch.data[0].as_in_context(ctx).reshape((-1,784))
            label = batch.label[0].as_in_context(ctx)
            label_one_hot = nd.one_hot(label, 10)
            output = net(data)
        
        metric.update([label], [output])
    return metric.get()[1]

## The Training Loop

In [118]:
epochs = 10
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        with autograd.record():
            data = batch.data[0].as_in_context(ctx).reshape((-1,784))
            label = batch.label[0].as_in_context(ctx)
            label_one_hot = nd.one_hot(label, 10)
            output = net(data)
            loss = cross_entropy(output, label_one_hot)
            loss.backward()
        SGD(params, .001)
        
        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = np.mean(loss.asnumpy()[0])
        else:
            moving_loss = .99 * moving_loss + .01 * np.mean(loss.asnumpy()[0])
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))    
    

Epoch 0. Loss: 0.450860380328, Train_acc 0.865368150685, Test_acc 0.868929140127
Epoch 1. Loss: 0.289160527269, Train_acc 0.893842751142, Test_acc 0.872591353834
Epoch 2. Loss: 0.227494174882, Train_acc 0.911334665145, Test_acc 0.897242490413
Epoch 3. Loss: 0.17750015641, Train_acc 0.923448202055, Test_acc 0.913436047356
Epoch 4. Loss: 0.136533414566, Train_acc 0.932308789954, Test_acc 0.924822983249
Epoch 5. Loss: 0.108078708505, Train_acc 0.939017313546, Test_acc 0.933277476918
Epoch 6. Loss: 0.0902558591182, Train_acc 0.944353392042, Test_acc 0.939694979188
Epoch 7. Loss: 0.076760384345, Train_acc 0.948783533105, Test_acc 0.944891012529
Epoch 8. Loss: 0.0676438058201, Train_acc 0.952449581431, Test_acc 0.949198160816
Epoch 9. Loss: 0.0561751382824, Train_acc 0.955552226027, Test_acc 0.952783222633


In [107]:
print("data shape: %s" % (np.array(data.shape)))

data shape: [ 64 784]


## Conclusion

Noice. With just two hidden layers containing 256 and 128 hidden nodes, repsectively, we can achieve over 95% accuracy on this task. 